In [1]:
# Re-run the entire XGBoost pipeline since the execution state was reset

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Reload dataset
file_path = "/mnt/data/epileptic_seizure_recognition.csv"
df = pd.read_csv(file_path)

# Rename last column to 'label'
df.rename(columns={df.columns[-1]: "label"}, inplace=True)

# Drop the first column which contains non-numeric values
df_cleaned = df.iloc[:, 1:]

# Separate features and labels
X = df_cleaned.iloc[:, :-1].values  # EEG features
y = df_cleaned["label"].values      # Labels

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert to 0,1,2

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y)

# Train XGBoost model
xgb_model = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, target_names=["Seizure Activity", "Tumor Area", "Healthy Region"], output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Display results
import ace_tools as tools
tools.display_dataframe_to_user(name="XGBoost Model Classification Report", dataframe=report_df)


ModuleNotFoundError: No module named 'xgboost'